In [50]:
import requests

In [51]:
standinds_url = "https://fbref.com/en/comps/12/2023-2024/2023-2024-La-Liga-Stats"

In [52]:
data = requests.get(standings_url)

In [53]:
from bs4 import BeautifulSoup

In [54]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [55]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [56]:
data = requests.get(team_urls[0])

In [57]:
import pandas as pd
from io import StringIO
matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

In [58]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [59]:
data = requests.get(f"https://fbref.com{links[0]}")

In [60]:
shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]

In [61]:
shooting.head()

For Real Madrid                                                       \
             Date   Time     Comp        Round  Day Venue Result GF GA   
0      2023-08-12  21:30  La Liga  Matchweek 1  Sat  Away      W  2  0   
1      2023-08-19  19:30  La Liga  Matchweek 2  Sat  Away      W  3  1   
2      2023-08-25  21:30  La Liga  Matchweek 3  Fri  Away      W  1  0   
3      2023-09-02  16:15  La Liga  Matchweek 4  Sat  Home      W  2  1   
4      2023-09-17  21:00  La Liga  Matchweek 5  Sun  Home      W  2  1   

                  ... Standard               Expected                    \
        Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0  Athletic Club  ...     16.0  2.0  0     0      0.9  0.9    0.07  1.1   
1        Almería  ...     17.0  1.0  0     0      2.0  2.0    0.08  1.0   
2     Celta Vigo  ...     19.4  0.0  0     1      1.4  0.6    0.06 -0.4   
3         Getafe  ...     17.7  0.0  0     0      2.8  2.8    0.11 -0.8   
4  Real Sociedad  ...     15.9  1.0  0     0      2.0  2.0    0.13  0.0   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0     1.1        Match Report  
1     1.0        Match Report  
2     0.4        Match Report  
3    -0.8        Match Report  
4     0.0        Match Report  

[5 rows x 26 columns]

In [62]:
shooting.columns = shooting.columns.droplevel()

In [63]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [64]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-12,21:30,La Liga,Matchweek 1,Sat,Away,W,2,0,Athletic Club,...,4-2-3-1,Jesús Gil,Match Report,NaN,14,8,16.0,2.0,0,0
1,2023-08-19,19:30,La Liga,Matchweek 2,Sat,Away,W,3,1,Almería,...,4-2-3-1,José Sánchez,Match Report,NaN,25,9,17.0,1.0,0,0
2,2023-08-25,21:30,La Liga,Matchweek 3,Fri,Away,W,1,0,Celta Vigo,...,5-3-2,Isidro Díaz de Mera,Match Report,NaN,9,2,19.4,0.0,0,1
3,2023-09-02,16:15,La Liga,Matchweek 4,Sat,Home,W,2,1,Getafe,...,4-1-3-2,Mario Melero,Match Report,NaN,26,12,17.7,0.0,0,0
4,2023-09-17,21:00,La Liga,Matchweek 5,Sun,Home,W,2,1,Real Sociedad,...,4-3-3,César Soto,Match Report,NaN,17,8,15.9,1.0,0,0


In [65]:
years = list(range(2022, 2015, -1))
all_matches = []

In [66]:
standings_url = "https://fbref.com/en/comps/12/2023-2024/2023-2024-La-Liga-Stats"

In [67]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(StringIO(data.text))
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        time.sleep(3)
        data = requests.get(team_url)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        soup = BeautifulSoup(StringIO(data.text))
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        time.sleep(3)
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "La Liga"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(3)

<ipython-input-67-fe0070441a54>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Season"] = year
<ipython-input-67-fe0070441a54>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_data["Team"] = team_name
<ipython-input-67-fe0070441a54>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [76]:
len(all_matches)

140

In [77]:
match_df = pd.concat(all_matches)

In [78]:
match_df.columns = [c.lower() for c in match_df.columns]

In [79]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-12,21:30,La Liga,Matchweek 1,Sat,Away,W,2,0,Athletic Club,...,Match Report,NaN,14.0,8.0,16.0,2.0,0,0,2022,Real Madrid
1,2023-08-19,19:30,La Liga,Matchweek 2,Sat,Away,W,3,1,Almería,...,Match Report,NaN,25.0,9.0,17.0,1.0,0,0,2022,Real Madrid
2,2023-08-25,21:30,La Liga,Matchweek 3,Fri,Away,W,1,0,Celta Vigo,...,Match Report,NaN,9.0,2.0,19.4,0.0,0,1,2022,Real Madrid
3,2023-09-02,16:15,La Liga,Matchweek 4,Sat,Home,W,2,1,Getafe,...,Match Report,NaN,26.0,12.0,17.7,0.0,0,0,2022,Real Madrid
4,2023-09-17,21:00,La Liga,Matchweek 5,Sun,Home,W,2,1,Real Sociedad,...,Match Report,NaN,17.0,8.0,15.9,1.0,0,0,2022,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2018-04-22,16:15,La Liga,Matchweek 34,Sun,Home,W,2,0,Real Sociedad,...,Match Report,NaN,5.0,4.0,11.5,0.0,0,0,2016,Malaga
36,2018-04-30,21:00,La Liga,Matchweek 35,Mon,Away,L,1,2,Betis,...,Match Report,NaN,9.0,3.0,16.7,0.0,0,0,2016,Malaga
37,2018-05-06,12:00,La Liga,Matchweek 36,Sun,Home,L,0,3,Alavés,...,Match Report,NaN,16.0,4.0,15.4,1.0,0,0,2016,Malaga
38,2018-05-13,16:15,La Liga,Matchweek 37,Sun,Away,L,1,4,Espanyol,...,Match Report,NaN,8.0,2.0,17.3,0.0,1,1,2016,Malaga


In [80]:
match_df.to_csv("laLigaData.csv")

In [81]:
import os
print(os.getcwd())

/content


In [82]:
matches

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Match Report,Notes
0,2017-08-21,22:00,La Liga,Matchweek 1,Mon,Home,L,0,1,Eibar,0.9,1.4,54.0,24753,Recio,3-4-3,4-4-2,David Medié,Match Report,NaN
1,2017-08-26,20:15,La Liga,Matchweek 2,Sat,Away,L,0,1,Girona,0.4,1.4,55.0,9966,Recio,4-2-3-1,3-4-3,Santiago Jaime,Match Report,NaN
2,2017-09-11,21:00,La Liga,Matchweek 3,Mon,Home,L,1,3,Las Palmas,1.0,2.0,50.0,22697,Recio,4-2-3-1,4-2-3-1,Xavi Estrada,Match Report,NaN
3,2017-09-16,20:45,La Liga,Matchweek 4,Sat,Away,L,0,1,Atlético Madrid,0.6,1.5,34.0,63114,Recio,4-2-3-1,4-4-2,José Sánchez,Match Report,NaN
4,2017-09-19,20:00,La Liga,Matchweek 5,Tue,Away,L,0,5,Valencia,0.8,3.0,51.0,36215,Recio,4-1-4-1,4-4-2,Alfonso Álvarez,Match Report,NaN
5,2017-09-23,18:30,La Liga,Matchweek 6,Sat,Home,D,3,3,Athletic Club,2.2,1.9,46.0,23425,Recio,4-5-1,4-2-3-1,Carlos del Cerro,Match Report,NaN
6,2017-09-30,16:15,La Liga,Matchweek 7,Sat,Away,L,0,2,Sevilla,0.4,2.2,37.0,34430,Recio,5-3-2,4-3-3,Daniel Trujillo,Match Report,NaN
7,2017-10-15,18:30,La Liga,Matchweek 8,Sun,Home,L,0,2,Leganés,1.0,2.2,54.0,23951,Recio,4-2-3-1,4-2-3-1,José González,Match Report,NaN
8,2017-10-21,20:45,La Liga,Matchweek 9,Sat,Away,L,0,2,Barcelona,0.6,1.9,35.0,74397,Roberto Rosales,4-2-3-1,4-4-2,Pablo González,Match Report,NaN
9,2017-10-24,19:30,Copa del Rey,Round of 32,Tue,Away,L,1,2,Numancia,NaN,NaN,NaN,3403,NaN,NaN,NaN,Alfonso Álvarez,Match Report,Leg 1 of 2
